In [95]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver import ChromeOptions
import time
import re

In [88]:
option = ChromeOptions()
option.add_argument('user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
browser=webdriver.Chrome(executable_path="/Applications/Google Chrome.app/Contents/MacOS/chromedriver", options=option)

In [89]:
browser.get("https://www.damai.cn/?utm_source=search&utm_medium=baidupc&utm_content=pmarket_yhcpyyb&utm_campaign=alink_bdsem_shh_shouye_8000")

In [90]:
def is_element_exist(query_type, value):
    """根据查询类型判断页面中是否存在相应的元素"""
    try:
        if query_type == "css":
            if len(browser.find_elements_by_css_selector(value)) > 0:
                return True
            else:
                return False
        elif query_type == "class":
            if len(browser.find_elements_by_class_name(value)) > 0:
                return True
            else:
                return False
        elif query_type == "id":
            if len(browser.find_elements_by_id(value)) > 0:
                return True
            else:
                return False
        elif query_type == "xpath":
            if len(browser.find_elements_by_xpath(value)) > 0:
                return True
            else:
                return False
        else:
            return "query type is error, only support css, class, id, xpath!!!"
    except:
        return False

In [91]:
def login():
    """登录"""
    """大麦会识别出非人工操作，从输入用户名密码开始到验证码结束人工操作"""
    #点击登录按钮,跳转到登录界面
    if browser.title=="大麦网-全球演出赛事官方购票平台-100%正品、先付先抢、在线选座！":
        browser.find_element_by_css_selector(".login-user").click()
        browser.implicitly_wait(10)
        #进入登录的iframe，坑，竟然在一个大页面中有一个iframe
        browser.switch_to_frame("alibaba-login-box")
    browser.find_element_by_id("fm-login-id").clear()
    #输入用户名，需要替换成自己的用户名
    browser.find_element_by_id("fm-login-id").send_keys("")
    browser.find_element_by_id("fm-login-password").clear()
    #输入密码
    browser.find_element_by_id("fm-login-password").send_keys("")
    #判断是否存在滑动解锁
    if is_element_exist("class", "nc-lang-cnt"):
        #滑动验证码,获取平移距离,移动解锁
        start=browser.find_element_by_id("nc_1_n1z")
        inner_size=start.size["width"]
        outer_size=browser.find_element_by_class_name("nc-lang-cnt").size["width"]
        move_size=outer_size-inner_size
        time.sleep(1)
        ActionChains(browser).drag_and_drop_by_offset(start, move_size, 0).perform()
        time.sleep(1)#滑完了之后稍等下，让系统判断完毕

    browser.find_element_by_css_selector(".password-login").click()

In [92]:
login()

/Users/admin/Software/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: use driver.switch_to.frame instead
  if __name__ == '__main__':


In [ ]:
def search(performer, city=None):
    """搜索"""
    if city != None:
        browser.find_element_by_link_text(city).click()
        browser.implicitly_wait(20)
    browser.find_element_by_class_name("input-search").send_keys(performer)
    browser.find_element_by_class_name("btn-search").click()
    browser.implicitly_wait(20)

In [94]:
def bug_detail(city):
    """进入购买详情页"""
    browser.find_element_by_partial_link_text(city).click()
    browser.implicitly_wait(20)

ActionChains	 ChromeOptions	 browser	 inner_size	 is_element_exist	 login	 move_size	 option	 outer_size	 
start	 time	 webdriver	 


In [ ]:
def buy_now(price, total, start_time):
    """
    当时间到达售票时间时，刷新页面，开始买票
    price为购买价格，total为购买票的数目，start_time为开票时间
    """
    browser.find_element_by_link_text(str(price) + "元").click()
    limit = re.search("\d",browser.find_element_by_class_name("number_right_limit").text).group(0)
    if total > limit:
        total=limit
    browser.find_element_by_class_name("cafe-c-input-number-input").send_keys(total)
    browser.find_element_by_class_name("buybtn").click()
    browser.implicitly_wait(10)

In [ ]:
def submit_order():
    """提交订单"""
    browser.find_element_by_class_name("next-checkbox-label").click()
    browser.find_elements_by_class_name("next-btn-medium")[2].click()
    
    # send email